In [1]:
import boto3
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
from folium import GeoJson
from io import BytesIO

In [2]:
import sys
sys.path.append("/agrilearn_app/agrilearn/submodules/commons/")

In [3]:
from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

from agrilearn.crop_classification import evalutate_utils
from agrilearn.crop_classification import yaml_utils
from agrilearn.commons.s3 import s3_utils

2025-03-06 22:45:21.694619: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 22:45:21.711487: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741301121.730793   12134 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741301121.736781   12134 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 22:45:21.756588: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

### Resumo da análise

- Existem colunas duplicadas que representam a mesma variável [``start_season``, ``start_of_season``], além de [``end_season``, ``end_of_season``], [``peak_of_season``, ``peak_season``]
- Existem dados duplicados considerando o ``eopatch_location``
- Não existe mais a coluna ``id_talhao``?

### Global Variables

In [4]:
GEOPACKAGE_PATH = ["/agrilearn_app/datasets/teste_pre_safra_2024_2025/geopackage/raw/bb_areas_and_teste_concordancia_SOYBEAN_5916_CORN_180_COTTON_51_PASTURE_50_SUGAR_CANE_50_WHEAT_50_RICE_19.gpkg"]

label_monitoring_class = 'monitoring_class'
label_eopatch_path = "eopatch_location" # 'sentinel_eopatch_current'

# 1. Read Datasets

### Data description
- monitoring_class: é o ``conjunto`` que representa a cultura de interesse do cliente ou da amostra de treinamento (SOYBEAN, CORN)
- period: é o ``texto`` que representa a safra agricola (2023/2024, 2024/2025)
- fonte: é o ``texto`` da origem dos dados (mapas temáticos, banco de dados)
- state: é o ``texto`` do estado do polígono
- area: é o ``ponto flutuante`` da área em métros
- micro: é o ``inteiro`` que representa a micro região
- start_season: é o ``timestamp`` que inicia a safra pra determinada cultura (monitoring_class) e safra (period)
- end_season: é o  ``timestamp`` que termina a safra pra determinada cultura (monitoring_class) e safra (period)
- peak_start: é o ``timestamp`` que inicia o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period)
- peak_end: é o ``timestamp`` que termina o período em que pode ocorrer o pico pra determinada cultura (monitoring_class) e safra (period) (Obs: temos um calendário pra isso, um arquivo yaml)
- eopath_location: é o ``texto`` que representa o caminho onde o eopatch está salvo (imagens p/ inferência)
- start_of_season: é o ``timestamp`` da data da emergencia da cultura
- peak_of_season: é o ``timestamp`` da data do pico vegetativo da cultura
- end_of_season: é o ``timestamp`` da data da colheita da cultura
- length_of_season: é o ``inteiro`` com a duração do cultivo em dias
- is_valid_metrics:
- sos_valid:
- pos_valid:
- eos_valid:
- los_valid:
- planting_start: é o ``timestamp`` da data do inicio do plantio.
- planting_end: é o ``timestamp`` da data do fim do plantio.
- start_of_cycle: é o ``timestamp`` da data do inio do cultivo segundo o calendário agrícola
- end_of_cycle: é o ``timestamp`` da data do fim do caledário de cultivo segundo calendário agrícola
- length_of_cycle: é o ``inteiro`` que representa o tamanho do ciclo em dias
- is_valid:
- is_valid_POS:
- is_valid_LOS:
- set_type: é o ``conjunto`` que representar a divisão do dado entre treino, validação e teste
- sampled_date: é o ``texto`` com a data do sample em mês e ano
- cultura_2:
- geometry: é a ``geometria`` do polígono 

In [5]:
gdfs = []

# Lê cada Geopackage e adiciona o GeoDataFrame à lista
for path in GEOPACKAGE_PATH:
    gdf = gpd.read_file(path)
    gdf['dataset_source'] = path  # Adiciona a coluna de origem
    gdfs.append(gdf)

# Concatena todos os GeoDataFrames em um único GeoDataFrame
if gdfs:
    df = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
    print(f"Dataset final possui {df.shape[0]} linhas")
    df.head()
else:
    print("Nenhum GeoDataFrame válido encontrado.")

Dataset final possui 6316 linhas


In [7]:
datetime_columns = ['start_season', 
                    'end_season', 
                    'peak_start', 
                    'peak_end',
                    'start_of_season',
                    'peak_of_season',
                    'end_of_season',
                    'planting_start',
                    'planting_end',
                    'start_of_cycle',
                    'end_of_cycle'                  
                   ]

    # Verifica se a coluna existe, se existir formata, senão printa mensagem
for col in datetime_columns:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
        except Exception as e:
            print(f"Erro ao formatar a coluna {col}: {e}")
    else:
        print(f"A coluna '{col}' não existe no DataFrame.")
        # Exibe informações das colunas datetime

A coluna 'peak_of_season' não existe no DataFrame.
A coluna 'start_of_cycle' não existe no DataFrame.
A coluna 'end_of_cycle' não existe no DataFrame.


In [8]:
df[label_monitoring_class].value_counts()

monitoring_class
SOYBEAN    6188
CORN        128
Name: count, dtype: int64

In [9]:
df['state'].value_counts()

state
RS    1398
GO    1255
MT    1071
MS     715
SP     557
BA     383
PR     278
TO     237
PA     164
MG     111
PI      66
MA      49
SC      21
RO       9
DF       1
SE       1
Name: count, dtype: int64

In [10]:
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")

There are 6316 rows and 76 columns


In [11]:
print(f"Dados de {df['start_season'].min()} a {df['end_season'].max()}")

Dados de 2019-10-01 00:00:00 a 2024-09-01 00:00:00


In [12]:
list(df.columns)

['interest_area_id',
 'period',
 'start_season',
 'no_till_farming',
 'emergence_date',
 'emergence_sensor',
 'gt_class',
 'senescence_date',
 'senescence_sensor',
 'harvest_date',
 'harvest_sensor',
 'end_season',
 'area_ha',
 'created_at',
 'modified_by',
 'modified_at',
 'zarc_planting',
 'zarc_emergence',
 'interest_area_code',
 'monitoring_class',
 'state',
 'peak_start',
 'peak_end',
 'sentinel_eopatch_current',
 'field_id',
 'contour_selected_timestamp',
 'contour_score',
 'obs_contour',
 'obs_contour_score',
 'agricultural_area',
 'score_farm_plot',
 'obs_farm_plot',
 'field_confidence',
 'field_score',
 'emergence_score',
 'obs_emergence',
 'center_x',
 'center_y',
 'crop_valid_cls',
 'senescence_peak_date',
 'obs_senescence',
 'harvest_confidence',
 'harvest_score',
 'obs_harvest',
 'obs_extra',
 'created_by',
 'area_id',
 'last_date_crop_rnn',
 'crop_distance_score_sits',
 'crop_confidence_maha_sits',
 'id',
 'set',
 'fonte',
 'cultura_2',
 'area',
 'meso',
 'eopath_location

# 02. Data Integration in columns

In [13]:
df[['gt_class', 'monitoring_class']].sample(10)

,gt_class,monitoring_class
3717,SOYBEAN,SOYBEAN
2327,SOYBEAN,SOYBEAN
5288,SOYBEAN,SOYBEAN
3804,SOYBEAN,SOYBEAN
5639,SOYBEAN,SOYBEAN
4969,SOYBEAN,SOYBEAN
5867,CORN,SOYBEAN
6191,SOYBEAN,SOYBEAN
3229,SOYBEAN,SOYBEAN
1254,SOYBEAN,SOYBEAN


In [16]:
df['monitoring_class'] = df['gt_class']
df[label_eopatch_path] = df['sentinel_eopatch_current']

In [ ]:
# # Atribui a coluna principal que são nulas os dados da coluna que tem nome diferente no dataset
# df['length_of_season'] = df['length_of_season'].fillna(df['LOS'])
# df['start_season'] = df['start_season'].fillna(df['start_of_season'])
# df['end_season'] = df['end_season'].fillna(df['end_of_season'])
# df['eopath_location'] = df['eopath_location'].fillna(df['sentinel_eopatch_current'])

# 03. Check Data Quality

## 3.1 Drop Columns NaN

In [17]:
shape_before = df.shape[1]
df.dropna(axis=1, how='all', inplace=True)
print(f"Removed Columns: {shape_before - df.shape[1]}, Percentage: {(shape_before - df.shape[1]) / shape_before * 100:.2f}%")

Removed Columns: 29, Percentage: 37.66%


## 3.2 Check NaN Values

In [18]:
# Calcula a quantidade de registros nulos e a porcentagem de registros nulos
df_check_NaN = pd.concat([df.isna().sum(), df.isna().sum() / df.shape[0] * 100], axis=1)
df_check_NaN.columns = ['Null Count', 'NaN percentage']
df_check_NaN.sort_values('NaN percentage', ascending=False, inplace=True)

In [19]:
df_check_NaN.iloc[:30]

,Null Count,NaN percentage
id,6315,99.984167
no_till_farming,6306,99.841672
peak_date,6067,96.057631
uf,5972,94.553515
progress_status,5972,94.553515
uuid,5972,94.553515
harvest_date,1699,26.899937
harvest_sensor,1642,25.997467
senescence_date,406,6.428119
crop_valid_cls,346,5.478151


- Q1) Quais colunas são geradas por nós e quais são coletadas? 
- Q2) Qual a razão dos dados nulos?
- Q3) Qual o melhor conjunto de colunas eu devo usar para considerar dados duplicados uma vez que o field_id não está disponível para alguns datasets?

In [20]:
df_check_duplicated = df[df.duplicated(subset=['geometry', 'period'], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby(['geometry', 'period']).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,set,progress_status,uf,uuid,peak_date,dataset_part,dataset_source,geometry,eopatch_location,id_duplicado
1135,1135,2023/2024,2023-09-01,None,2023-10-30,ALTERNATIVE_DATA,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.9155 -10.03081, -51.91497 ...",start_2023-09-01_end_2024-05-01_monitoring_cla...,1
2600,2600,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.9155 -10.03081, -51.91497 ...",start_2023-09-01_end_2024-05-01_monitoring_cla...,1
2601,2601,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.909 -10.02116, -51.90899 -...",start_2023-09-01_end_2024-05-01_monitoring_cla...,2
1133,1133,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.909 -10.02116, -51.90899 -...",start_2023-09-01_end_2024-05-01_monitoring_cla...,2
5259,5259,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.90896 -10.03291, -51.90907...",start_2023-09-01_end_2024-05-01_monitoring_cla...,3
4024,4024,2023/2024,2023-09-01,None,2023-10-30,ALTERNATIVE_DATA,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.90896 -10.03291, -51.90907...",start_2023-09-01_end_2024-05-01_monitoring_cla...,3
5264,5264,2023/2024,2023-09-01,None,2023-10-26,OLI / Landsat 9,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-01-22,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.89492 -10.02655, -51.89412...",start_2023-09-01_end_2024-05-01_monitoring_cla...,4
4023,4023,2023/2024,2023-09-01,None,2023-10-26,OLI / Landsat 9,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-01-22,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.89492 -10.02655, -51.89412...",start_2023-09-01_end_2024-05-01_monitoring_cla...,4
4018,4018,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2023-12-19,MSI / Sentinel 2,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.90899 -10.02124, -51.90396...",start_2023-09-01_end_2024-05-01_monitoring_cla...,5
5261,5261,2023/2024,2023-09-01,None,2023-10-30,MSI / Sentinel 2,SOYBEAN,2024-01-01,MODIS / TERRA - AQUA,2024-02-07,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-51.90899 -10.02124, -51.90396...",start_2023-09-01_end_2024-05-01_monitoring_cla...,5


In [ ]:
# df_check_duplicated.to_csv('/agrilearn_app/datasets/teste_pre_safra_2024_2025/reports/report_duplicated_values_geometry_period_eopatch_location_bb_areas_and_teste_concordancia_SOYBEAN_5916_CORN_180_COTTON_51_PASTURE_50_SUGAR_CANE_50_WHEAT_50_RICE_19.csv',
#                            sep=';',
#                            decimal=',',
#                            float_format='%.4f')

In [21]:
shape_before = df.shape[0]
df.drop_duplicates(subset=['geometry', 'period'], inplace=True)
print(f"Removed records: {shape_before - df.shape[0]}, Percentage: {(shape_before - df.shape[0]) / shape_before * 100:.2f}%")

Removed records: 13, Percentage: 0.21%


# 04. Univariate Data Analysis

### 4.1 Analysing Target y ``monitoring_class``

In [22]:
df_check_NaN.loc['monitoring_class']

Null Count        0.0
NaN percentage    0.0
Name: monitoring_class, dtype: float64

In [23]:
df['monitoring_class'].nunique()

7

In [24]:
pd.concat([df['monitoring_class'].value_counts(),
          df['monitoring_class'].value_counts(normalize=True)*100], axis=1)

,count,proportion
monitoring_class,,
SOYBEAN,5903,93.653816
CORN,180,2.855783
COTTON,51,0.809139
PASTURE,50,0.793273
SUGAR_CANE,50,0.793273
WHEAT,50,0.793273
RICE,19,0.301444


### 4.2 Analysing dados por ``estado``

In [25]:
df_check_NaN.loc['state']

Null Count        0.0
NaN percentage    0.0
Name: state, dtype: float64

In [26]:
df['state'].nunique()

16

In [27]:
pd.concat([df['state'].value_counts(),
          df['state'].value_counts(normalize=True)*100], axis=1)

,count,proportion
state,,
RS,1398,22.179914
GO,1255,19.911153
MT,1058,16.785658
MS,715,11.343805
SP,557,8.837062
BA,383,6.076472
PR,278,4.410598
TO,237,3.760114
PA,164,2.601936


### 4.3 Analysis ``period``

In [28]:
df_check_NaN.loc['period']

Null Count        0.0
NaN percentage    0.0
Name: period, dtype: float64

In [29]:
df['period'].nunique()

9

In [30]:
pd.concat([df['period'].value_counts(),
          df['period'].value_counts(normalize=True)*100], axis=1)

,count,proportion
period,,
2023/2024,5959,94.542281
2020/2021,119,1.887990
2023/2023,83,1.316833
2022/2023,80,1.269237
2021/2021,42,0.666349
2021/2022,12,0.190386
2019/2020,5,0.079327
2022/2022,2,0.031731
2020/2020,1,0.015865


### 4.12 Analysis of ``eopath_location``

In [31]:
df_check_NaN.loc['eopatch_location']

Null Count        1.000000
NaN percentage    0.015833
Name: eopatch_location, dtype: float64

In [32]:
df.dropna(subset=[label_eopatch_path])

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,id,set,progress_status,uf,uuid,peak_date,dataset_part,dataset_source,geometry,eopatch_location
0,0,2023/2024,2023-10-01,None,2024-01-12,MSI / Sentinel 2,SOYBEAN,2024-03-04,MSI / Sentinel 2,None,...,NaN,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-49.87426 -17.49841, -49.87428...",start_2023-10-01_end_2024-05-01_monitoring_cla...
1,1,2023/2024,2023-10-01,None,2024-01-12,MSI / Sentinel 2,SOYBEAN,2024-03-04,MSI / Sentinel 2,None,...,NaN,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-49.87649 -17.50258, -49.88479...",start_2023-10-01_end_2024-05-01_monitoring_cla...
2,2,2023/2024,2023-10-01,None,2023-12-06,OLI / Landsat 9,SOYBEAN,2024-02-24,OLI / Landsat 9,2024-03-05,...,NaN,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-48.5898 -4.8188, -48.58982 -4...",start_2023-10-01_end_2024-09-01_monitoring_cla...
3,3,2023/2024,2023-10-01,None,2024-01-17,OLI / Landsat 8,SOYBEAN,2024-03-14,MSI / Sentinel 2,None,...,NaN,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.92268 -17.87173, -47.92267...",start_2023-10-01_end_2024-05-01_monitoring_cla...
4,4,2023/2024,2023-10-01,None,2024-01-17,OLI / Landsat 8,SOYBEAN,2024-03-13,OLI / Landsat 9,None,...,NaN,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.92267 -17.87151, -47.92268...",start_2023-10-01_end_2024-05-01_monitoring_cla...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,327,2023/2023,2023-02-01,None,2021-12-20,MSI / Sentinel 2,SUGAR_CANE,None,None,2022-11-29,...,NaN,run_2082_mvp_344_teste_concordancia_crop_120.gpkg,ENCERRADO,SP,{02a0f996-ba82-4adc-a417-dd350e73756e},None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.26655 -22.6615, -47.26652 ...",327_start_2023-02-01_end_2023-10-01_monitoring...
6311,328,2023/2023,2023-02-01,None,2021-11-25,MSI / Sentinel 2,SUGAR_CANE,None,None,2022-10-06,...,NaN,run_2082_mvp_344_teste_concordancia_crop_120.gpkg,ENCERRADO,SP,{822ff516-1caf-4c13-9385-4e4461397674},None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.25891 -22.61851, -47.25884...",328_start_2023-02-01_end_2023-10-01_monitoring...
6312,329,2023/2023,2023-02-01,None,2021-11-05,MSI / Sentinel 2,SUGAR_CANE,None,None,2022-11-25,...,NaN,run_2082_mvp_344_teste_concordancia_crop_120.gpkg,ENCERRADO,SP,{d15c8fd1-24b8-4643-9543-7d1d99047797},None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.24072 -22.66836, -47.24241...",329_start_2023-02-01_end_2023-10-01_monitoring...
6313,330,2023/2023,2023-02-01,None,2021-09-11,MSI / Sentinel 2,SUGAR_CANE,None,None,2022-11-05,...,NaN,run_2082_mvp_344_teste_concordancia_crop_120.gpkg,ENCERRADO,SP,{2e14dc3f-9e78-4cd1-96e3-34e821c75683},None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-47.23388 -22.64052, -47.235 -...",330_start_2023-02-01_end_2023-10-01_monitoring...


In [33]:
df[label_eopatch_path].nunique()

6295

In [34]:
df.shape[0]

6303

In [35]:
df_check_duplicated = df[df.duplicated(subset=[label_eopatch_path], keep=False)].copy()
df_check_duplicated['id_duplicado'] = (df_check_duplicated.groupby([label_eopatch_path]).ngroup() + 1)
df_check_duplicated.sort_values('id_duplicado', inplace=True)
df_check_duplicated

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,set,progress_status,uf,uuid,peak_date,dataset_part,dataset_source,geometry,eopatch_location,id_duplicado
428,428,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-28,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.34734 -21.49192, -56.34738...",start_2023-09-01_end_2024-05-01_monitoring_cla...,1
3504,3504,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-28,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.34738 -21.49142, -56.34753...",start_2023-09-01_end_2024-05-01_monitoring_cla...,1
429,429,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-24,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35173 -21.48316, -56.35169...",start_2023-09-01_end_2024-05-01_monitoring_cla...,2
3507,3507,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-28,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35169 -21.48316, -56.3504 ...",start_2023-09-01_end_2024-05-01_monitoring_cla...,2
3510,3510,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-28,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35312 -21.49631, -56.35351...",start_2023-09-01_end_2024-05-01_monitoring_cla...,3
432,432,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-18,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35296 -21.4962, -56.35312 ...",start_2023-09-01_end_2024-05-01_monitoring_cla...,3
430,430,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-24,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35565 -21.49684, -56.35449...",start_2023-09-01_end_2024-05-01_monitoring_cla...,4
3508,3508,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-24,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35449 -21.49754, -56.35399...",start_2023-09-01_end_2024-05-01_monitoring_cla...,4
433,433,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-18,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35202 -21.48513, -56.35213...",start_2023-09-01_end_2024-05-01_monitoring_cla...,5
3505,3505,2023/2024,2023-09-01,None,2023-12-24,MSI / Sentinel 2,SOYBEAN,2024-02-22,MSI / Sentinel 2,2024-03-16,...,run_2082_mvp_5972_bb_areas_crop_120.gpkg,None,None,None,None,test,/agrilearn_app/datasets/teste_pre_safra_2024_2...,"MULTIPOLYGON (((-56.35213 -21.48548, -56.3524 ...",start_2023-09-01_end_2024-05-01_monitoring_cla...,5


In [38]:
# Verifica se há duplicatas na coluna especificada
duplicates = df[label_eopatch_path].duplicated(keep=False)

# Calcula a quantidade de dados duplicados e o percentual em relação ao dataset final
total_rows = len(df)
duplicate_count = duplicates.sum()
duplicate_percentage = (duplicate_count / total_rows) * 100

print(f"Quantidade de dados duplicados: {duplicate_count}")
print(f"Percentual de dados duplicados: {duplicate_percentage:.2f}%")

Quantidade de dados duplicados: 14
Percentual de dados duplicados: 0.22%


* **Question**: O mesmo geopackage está apontando par ao mesmo eopatch_location? *

### 4.13 Check ``start_season`` and ``end_season``

In [39]:
df_check_NaN.loc[['start_season', 'end_season']]

,Null Count,NaN percentage
start_season,1,0.015833
end_season,1,0.015833


In [40]:
df[['start_season', 'end_season']].describe()

,start_season,end_season
count,6302,6302
mean,2023-08-11 20:13:06.036179200,2024-04-11 06:49:14.490637824
min,2019-10-01 00:00:00,2020-05-01 00:00:00
25%,2023-09-01 00:00:00,2024-05-01 00:00:00
50%,2023-10-01 00:00:00,2024-05-01 00:00:00
75%,2023-10-01 00:00:00,2024-06-01 00:00:00
max,2023-10-01 00:00:00,2024-09-01 00:00:00


In [41]:
df.groupby(df['start_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

,count
start_season,
2019.0,5
2020.0,120
2021.0,53
2022.0,82
2023.0,6042


In [42]:
df.groupby(df['end_season'].dt.year).agg(count=(label_monitoring_class, 'count'))

,count
end_season,
2020.0,6
2021.0,160
2022.0,14
2023.0,163
2024.0,5959


In [43]:
df[df['start_season'] == df['end_season']]

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,id,set,progress_status,uf,uuid,peak_date,dataset_part,dataset_source,geometry,eopatch_location


### 4.14 Check ``start_season`` and ``end_season``

In [44]:
df_check_NaN.loc[['peak_start', 'peak_end']]

,Null Count,NaN percentage
peak_start,1,0.015833
peak_end,1,0.015833


In [45]:
df[['peak_start', 'peak_end']].describe()

,peak_start,peak_end
count,6302,6302
mean,2023-10-11 19:15:17.423040512,2024-01-11 06:47:38.521104640
min,2019-12-01 00:00:00,2020-02-01 00:00:00
25%,2023-11-01 00:00:00,2024-02-01 00:00:00
50%,2023-12-01 00:00:00,2024-02-01 00:00:00
75%,2023-12-01 00:00:00,2024-03-01 00:00:00
max,2023-12-01 00:00:00,2024-06-01 00:00:00


In [46]:
df[['peak_start', 'peak_end']].describe()

,peak_start,peak_end
count,6302,6302
mean,2023-10-11 19:15:17.423040512,2024-01-11 06:47:38.521104640
min,2019-12-01 00:00:00,2020-02-01 00:00:00
25%,2023-11-01 00:00:00,2024-02-01 00:00:00
50%,2023-12-01 00:00:00,2024-02-01 00:00:00
75%,2023-12-01 00:00:00,2024-03-01 00:00:00
max,2023-12-01 00:00:00,2024-06-01 00:00:00


In [47]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

,count
peak_start,
2019.0,5
2020.0,120
2021.0,53
2022.0,82
2023.0,6042


In [48]:
df.groupby(df['peak_end'].dt.year).agg(count=(label_monitoring_class, 'count'))

,count
peak_end,
2020.0,6
2021.0,160
2022.0,14
2023.0,163
2024.0,5959


In [49]:
df[df['peak_start'] == df['peak_end']]

,interest_area_id,period,start_season,no_till_farming,emergence_date,emergence_sensor,gt_class,senescence_date,senescence_sensor,harvest_date,...,id,set,progress_status,uf,uuid,peak_date,dataset_part,dataset_source,geometry,eopatch_location


In [50]:
df['peak_start'].dt.year.unique()

array([2023., 2020., 2021., 2019., 2022.,   nan])

In [51]:
df['peak_end'].dt.year.unique()

array([2024., 2021., 2022., 2020., 2023.,   nan])

In [52]:
df.groupby(df['peak_start'].dt.year).agg(count=(label_monitoring_class, 'count'))

,count
peak_start,
2019.0,5
2020.0,120
2021.0,53
2022.0,82
2023.0,6042


# 5. Multivariate Analysis

In [57]:
# df = df[df['set'].isin(['run_2082_mvp_344_teste_concordancia_crop_120.gpkg', 'run_2082_mvp_5972_bb_areas_crop_120.gpkg'])]

In [58]:
df[label_monitoring_class].value_counts()

monitoring_class
SOYBEAN       5903
CORN           180
COTTON          51
PASTURE         50
SUGAR_CANE      50
WHEAT           50
RICE            19
Name: count, dtype: int64

In [59]:
columns_to_group = ['period', 'state', label_monitoring_class]
report_distribuition = df.groupby(columns_to_group).agg(count=(label_eopatch_path, 'count')).reset_index()

# Calcula o percentual em relação ao total
total_count = len(df)
report_distribuition['percentage'] = (report_distribuition['count'] / total_count) * 100

In [60]:
report_distribuition

,period,state,monitoring_class,count,percentage
0,2019/2020,MG,SUGAR_CANE,5,0.079327
1,2020/2020,PR,CORN,1,0.015865
2,2020/2021,GO,CORN,7,0.111058
3,2020/2021,GO,PASTURE,11,0.174520
4,2020/2021,GO,SOYBEAN,1,0.015865
...,...,...,...,...,...
77,2023/2024,RS,SOYBEAN,1252,19.863557
78,2023/2024,SC,SOYBEAN,19,0.301444
79,2023/2024,SP,CORN,6,0.095193
80,2023/2024,SP,SOYBEAN,475,7.536094


In [61]:
result_string = '_'.join([f"{cls}_{count}" for cls, count in df[label_monitoring_class].value_counts().items()])
result_string

'SOYBEAN_5903_CORN_180_COTTON_51_PASTURE_50_SUGAR_CANE_50_WHEAT_50_RICE_19'

In [62]:
report_distribuition.to_csv(f'/agrilearn_app/datasets/teste_pre_safra_2024_2025/reports/check_distribuition_by_class_and_state_{result_string}.csv',
    sep=';',
    decimal=',',
    float_format='%.4f',
)

In [ ]:
# df.to_file(f"/agrilearn_app/datasets/teste_pre_safra_2024_2025/geopackage/raw/bb_areas_and_teste_concordancia_{result_string}.gpkg", driver='GPKG', engine='fiona')